<a href="https://colab.research.google.com/github/BurgundyIsAPublicEnemy/ORLIAn/blob/main/Generate_Visualizations_Using_GradCam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generate a visualization using GradCam
This notebook allows you to generate visualizations for the Eln-AI-ny model

Usage: 
1. Once you have a model trained, configure the bottom lines (follow variable names) 
2. Hit run
3. Check FOLDER_TO_OUTPUT to see where your models have been outputted

In [1]:
!pip install colab-env --upgrade
!pip install pytorch-gradcam
!pip install pytorch_lightning

In [2]:
import colab_env

colab_env.__version__

Python-dotenv could not parse statement starting at line 7


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'0.2.0'

In [3]:
!more /content/gdrive/MyDrive/vars.env

IMAGEFILES = 'WHERE YOUR DATAIKU IMAGES ARE LOCATED'
TABULARTRAINING = 'WHERE THE TRAINNG CSV IS AT'
TABULARTEST = 'WHERE THE TEST CSV IS AT '
FULLSTACKTRAIN = 'WHERE YOUR TIFS ARE'
TRAIN_AUGMENTED_PATH = 'WHERE YOU WANT TO PUT YOUR NEW TRAINING CSV'
TEST_AUGMENTED_PATH = 'WHERE YOU WANT TO PUT YOUR NEW TEST CSV'
METADATA_PATH = 'THE PARENT FOLDER FOR TRAIN_AUGMENTED_PATH AND TEST_AUGMENTED_P
ATH - ENDS WITH A /
TENPERCENT_MODEL_PATH = 'WHERE TEN PERCENT IS STORED'
IMG_TO_ANALYZE = 'WHAT IMAGE YOU WANT TO RUN VISUALIZATIONS ON'
FOLDER_TO_OUTPUT = 'THE FOLDER YOU WANT TO TO STORE YOUR IMAGE IN'
MODEL_PATH = 'OUTPUT YOUR TRAINED MODELS'
patience = 'How long you want to be patient before early stopping'


In [5]:
import os

TENPERCENT_MODEL_PATH = os.getenv("TENPERCENT_MODEL_PATH")
IMG_TO_ANALYZE = os.getenv("IMG_TO_ANALYZE")
FOLDER_TO_OUTPUT = os.getenv("FOLDER_TO_OUTPUT")
MODEL_PATH = os.getenv("MODEL_PATH")

In [7]:
print(TENPERCENT_MODEL_PATH)

WHAT IMAGE YOU WANT TO RUN VISUALIZATIONS ON


Set up imports

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, utils
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
from PIL import Image
import json
import cv2
import sys
%matplotlib inline

Set up model to load Eln-AI-ny back in

In [ ]:
class Global_Model(nn.Module): # model to combina both tab and imag 
    def __init__(self , pretrained_medical_resnet):
        super().__init__()
        self.conv_model = pretrained_medical_resnet # after freezing some layers and setting regression node as 1 not 4 
        self.tab_model = nn.Sequential(nn.Linear(13, 500),
                                  nn.BatchNorm1d(500),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2),
                                  nn.Linear(500,250),
                                  nn.BatchNorm1d(250),
                                  nn.ReLU(),
                                  nn.Dropout(p=0.2))
        self.output = nn.Linear(256 + 250 , 1 , bias = True) # addition of output of concat for image  and tab then have final output 
    
    def forward(self, x,tab): # x image , tab is tabular data related 
        x = self.conv_model(x)
        tab_out = self.tab_model(tab)
        x = torch.cat([x, tab_out],dim=1)
        return self.output(x)

    def saveConvParams(self): 
        torch.save(self.conv_model.state_dict(), f'')

    def getConv(self): 
            return self.conv_model

## Redownload 10Percent

Credit: https://github.com/ozanciga/self-supervised-histopathology/tree/tenpercent

Paper: https://arxiv.org/pdf/2011.13971.pdf

In [ ]:
RETURN_PREACTIVATION = False  # return features from the model, if false return classification logits
NUM_CLASSES = 1  # only used if RETURN_PREACTIVATION = False


def load_model_weights(model, weights):

    model_dict = model.state_dict()
    weights = {k: v for k, v in weights.items() if k in model_dict}
    if weights == {}:
        print('No weight could be loaded..')
    model_dict.update(weights)
    model.load_state_dict(model_dict)

    return model


model = torchvision.models.__dict__['resnet18'](pretrained=False)
state = torch.load(TENPERCENT_MODEL_PATH, map_location='cuda:0')

state_dict = state['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace('model.', '').replace('resnet.', '')] = state_dict.pop(key)

model = load_model_weights(model, state_dict)

if RETURN_PREACTIVATION:
    model.fc = torch.nn.Sequential()
else:
    model.fc = torch.nn.Linear(model.fc.in_features, NUM_CLASSES)

model = model.cuda()

images = torch.rand((10, 3, 224, 224), device='cuda')

out = model(images)


for param in model.parameters():
    param.requires_grad = False
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
    
    model.fc = torch.nn.Linear(512, 256,bias = True) 
    model = model.cuda()


## Load your model weights

In [ ]:
import copy

proposed_model = copy.deepcopy(model)

proposed_model = proposed_model.cuda()

model_device = torch.device('cpu')

model = copy.deepcopy(Global_Model(copy.deepcopy(proposed_model))) 

model.load_state_dict(torch.load(str(MODEL_PATH) , map_location = model_device), strict=False)

_IncompatibleKeys(missing_keys=['output.weight', 'output.bias'], unexpected_keys=['output.0.weight', 'output.0.bias', 'output.3.weight', 'output.3.bias', 'output.6.weight', 'output.6.bias', 'output.9.weight', 'output.9.bias', 'output.12.weight', 'output.12.bias'])

## Set up visualization functions 
Credit and forked to work with Regressors from: https://github.com/jacobgil/pytorch-explain-black-box

Original paper: https://arxiv.org/abs/1704.03296

In [ ]:
use_cuda = True
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

def tv_norm(input, tv_beta):
	img = input[0, 0, :]
	row_grad = torch.mean(torch.abs((img[:-1 , :] - img[1 :, :])).pow(tv_beta))
	col_grad = torch.mean(torch.abs((img[: , :-1] - img[: , 1 :])).pow(tv_beta))
	return row_grad + col_grad

def preprocess_image(img):
	means=[0.485, 0.456, 0.406]
	stds=[0.229, 0.224, 0.225]

	preprocessed_img = img.copy()[: , :, ::-1]
	for i in range(3):
		preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
		preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
	preprocessed_img = \
		np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))

	if use_cuda:
		preprocessed_img_tensor = torch.from_numpy(preprocessed_img).cuda()
	else:
		preprocessed_img_tensor = torch.from_numpy(preprocessed_img)

	preprocessed_img_tensor.unsqueeze_(0)
	return Variable(preprocessed_img_tensor, requires_grad = False)

def save(mask, img, blurred, img_name, folder):
	mask = mask.cpu().data.numpy()[0]
	mask = np.transpose(mask, (1, 2, 0))

	mask = (mask - np.min(mask)) / np.max(mask)
	mask = 1 - mask
	heatmap = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
	
	heatmap = np.float32(heatmap) / 255
	cam = 1.0*heatmap + np.float32(img)/255
	cam = cam / np.max(cam)

	img = np.float32(img) / 255
	perturbated = np.multiply(1 - mask, img) + np.multiply(mask, blurred)	
 
	cv2.imwrite(folder + img_name.split('/')[-1] + "_perturbated.png", np.uint8(255*perturbated))
	cv2.imwrite(folder + img_name.split('/')[-1] + "_heatmap.png", np.uint8(255*heatmap))
	cv2.imwrite(folder + img_name.split('/')[-1] + "_mask.png", np.uint8(255*mask))
	cv2.imwrite(folder + img_name.split('/')[-1] + "_cam.png", np.uint8(255*cam))

def numpy_to_torch(img, requires_grad = True):
	if len(img.shape) < 3:
		output = np.float32([img])
	else:
		output = np.transpose(img, (2, 0, 1))

	output = torch.from_numpy(output)
	if use_cuda:
		output = output.cuda()

	output.unsqueeze_(0)
	v = Variable(output, requires_grad = requires_grad)
	return v

def genGradCam(img_name, model, folder):
  tv_beta = 3
  learning_rate = 0.1
  max_iterations = 500
  l1_coeff = 0.01
  tv_coeff = 0.2

  cnn_model = model.conv_model
  original_img = cv2.imread(img_name)
  original_img = cv2.resize(original_img, (224, 224))
  img = np.float32(original_img) / 255
  blurred_img1 = cv2.GaussianBlur(img, (11, 11), 5)
  blurred_img2 = np.float32(cv2.medianBlur(original_img, 11))/255
  blurred_img_numpy = (blurred_img1 + blurred_img2) / 2
  mask_init = np.ones((28, 28), dtype = np.float32)

  # Convert to torch variables
  img = preprocess_image(img)
  blurred_img = preprocess_image(blurred_img2)
  mask = numpy_to_torch(mask_init)

  if use_cuda:
    upsample = torch.nn.UpsamplingBilinear2d(size=(224, 224)).cuda()
  else:
    upsample = torch.nn.UpsamplingBilinear2d(size=(224, 224))
  optimizer = torch.optim.Adam([mask], lr=learning_rate)

  target = torch.nn.ReLU()(cnn_model(img))
  category = np.argmax(target.cpu().data.numpy())
  print("Category with highest probability", category)
  print( "Optimizing.. ")

  for i in range(max_iterations):
    upsampled_mask = upsample(mask)
    # The single channel mask is used with an RGB image, 
    # so the mask is duplicated to have 3 channel,
    upsampled_mask = \
      upsampled_mask.expand(1, 3, upsampled_mask.size(2), \
                    upsampled_mask.size(3))
    
    # Use the mask to perturbated the input image.
    perturbated_input = img.mul(upsampled_mask) + \
              blurred_img.mul(1-upsampled_mask)
    
    noise = np.zeros((224, 224, 3), dtype = np.float32)
    cv2.randn(noise, 0, 0.2)
    noise = numpy_to_torch(noise)
    perturbated_input = perturbated_input + noise
    
    outputs = torch.nn.ReLU()(cnn_model(perturbated_input))
    loss = l1_coeff*torch.mean(torch.abs(1 - mask)) + \
        tv_coeff*tv_norm(mask, tv_beta) + outputs[0, category]

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Optional: clamping seems to give better results
    mask.data.clamp_(0, 1)

  upsampled_mask = upsample(mask)
  save(upsampled_mask, original_img, blurred_img_numpy, img_name, folder)

## Run and output CAMS

In [ ]:
img_name = IMG_TO_ANALYZE
genGradCam(img_name, model, FOLDER_TO_OUTPUT)

'\nimg_name = \'/content/drive/MyDrive/Epidemium Season 3/ORLIAn /Analysis/8aab52_[7778,34531]_component_data.tif.jpg\'\n#img_name = \'/content/drive/MyDrive/Epidemium Season 3/ORLIAn /Analysis/8b44cd_[13491,59805]_component_data.tif.jpg\'\ngenGradCam(img_name, model, "/content/drive/MyDrive/Epidemium Season 3/ORLIAn /Outputs/OtherModels/")\n'